# Affiliate Summit West 2025 Lead Scraper

In [101]:
import requests
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

## Load HTML

In [102]:
df = pd.read_csv("Leads_Data_with_Profile_Links.csv")

In [103]:
base = "https://affiliatesummit.app.swapcard.com"
prof = df.iloc[0]["Profile Link"]
full = base + prof

In [104]:
df["Full Link"] = base + df["Profile Link"]

## Glimpse Data

In [105]:
df

,Name,Job Title,Company,Profile Link,Full Link
0,Rob ! Caiello,VP Internet Marketplace Platforms,Allconnect (Red Ventures),/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
1,Zach ! Fertig,Owner,Property Leads (Liz Buys Houses),/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
2,Zach . Allen,Media Buyer,MDRN Remedies,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
3,Laz . Chavez,President,Gorilla Marketers Inc,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
4,Thomas . Rogers,Media buyer/ Affiliates,Connect Consulting,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
...,...,...,...,...,...
5107,Ofer Zur,CEO,Legal Harbor Credit Repair,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
5108,Diego Zurbaran,Director of Operations,HEALE Labs,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
5109,Amy Zuspan,Account Director,Atolls,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...
5110,Gabrielle Zuvic,Media Buyer,Taborlin Media Inc.,/event/affiliate-summit-west-2025/person/RXZlb...,https://affiliatesummit.app.swapcard.com/event...


In [107]:
df.to_csv("leads_links.csv")

## Launch Selenium

In [99]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)

In [44]:
driver.get(base + df.iloc[0]["Profile Link"])

In [68]:
with open("html.txt", "w") as f:
    f.write(driver.page_source)

## Iterate Through Eeach Lead Profile, Build Enriched DataFrame

In [100]:
# List to store extracted profile data
profile_urls = df["Full Link"]

all_profiles_data = []

# Iterate through the first five profiles
for url in profile_urls:
    driver.get(url)
    time.sleep(5)  # Wait for the page to load fully

    # Get the HTML content and parse with BeautifulSoup
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    # Dictionary to store extracted data
    profile_data = {
        "Profile URL": url,
        "About Me": "NA",
        "Attendee Type": "NA",
        "Lead Generation Business": "NA",
        "Vertical": "NA",
        "Company Size": "NA",
        "Solutions Needed": "NA",
        "Objective For Attending": "NA",
        "Social Media": "NA",
        "Contact Details": "NA"
    }

    # Extract "About Me" section
    about_me_section = soup.find("h2", string="About me")
    if about_me_section:
        about_me_text = about_me_section.find_next("div").get_text(strip=True)
        profile_data["About Me"] = about_me_text

    # Extract profile details such as "Attendee Type", "Lead Generation Business", etc.
    for detail_div in soup.find_all("div", class_="sc-45c084b0-0 ffSNIm"):
        label_div = detail_div.find("div", class_="sc-45c084b0-1 RuEbB")
        value_div = detail_div.find("div", class_="sc-45c084b0-2 jsVWbR")

        if label_div and value_div:
            label = label_div.get_text(strip=True)
            values = [span.get_text(strip=True) for span in value_div.find_all("span")]
            profile_data[label] = ", ".join(values) if values else value_div.get_text(strip=True)

    # Extract Social Media link
    social_media_section = soup.find("h2", string="Social media")
    if social_media_section:
        social_media_link = social_media_section.find_next("a", href=True)
        if social_media_link:
            profile_data["Social Media"] = social_media_link["href"]

    # Extract Contact Details link
    contact_details_section = soup.find("h2", string="Contact details")
    if contact_details_section:
        contact_link = contact_details_section.find_next("a", href=True)
        if contact_link:
            profile_data["Contact Details"] = contact_link["href"]

    # Append extracted profile data to the list
    all_profiles_data.append(profile_data)

# Convert data to a Pandas DataFrame
df = pd.DataFrame(all_profiles_data)

# Save to CSV for further analysis
df.to_csv("all_profiles_final.csv", index=False)

df

,Profile URL,About Me,Attendee Type,Lead Generation Business,Vertical,Company Size,Solutions Needed,Objective For Attending,Social Media,Contact Details,Affiliate Type
0,https://affiliatesummit.app.swapcard.com/event...,If you would like to white label Allconnect’s ...,Network/SaaS,NA,NA,1 (it's just me),NA,NA,NA,NA,NaN
1,https://affiliatesummit.app.swapcard.com/event...,We are a direct advertiser ($20m/year in spend...,Retailer/Brand,We buy leads,Home & Garden,5 to 10,"Advertising - Google Ads, Advertising - Native...",To find new publishers / affiliates,NA,NA,NaN
2,https://affiliatesummit.app.swapcard.com/event...,I specialize in creating brands and driving gr...,NA,NA,NA,NA,NA,NA,NA,NA,NaN
3,https://affiliatesummit.app.swapcard.com/event...,Gorilla Marketers is a direct media-buying com...,Affiliate/Publisher,NA,NA,11 to 25,NA,NA,skype://papi.laz?chat,https://gorillamarketers.com/,Media Buyer
4,https://affiliatesummit.app.swapcard.com/event...,I specialize in creating brands and driving gr...,Affiliate/Publisher,NA,NA,2 to 4,NA,To network,NA,NA,Media Buyer
...,...,...,...,...,...,...,...,...,...,...,...
5107,https://affiliatesummit.app.swapcard.com/event...,We looking after new sources for our credit re...,Retailer/Brand,We buy leads,Financial Services,26 to 50,Lead Generation - General,To find new publishers / affiliates,https://www.linkedin.com/in/http://linkedin.co...,https://www.legalharbor.com,NaN
5108,https://affiliatesummit.app.swapcard.com/event...,Attendee TypeEcommerce Store,Ecommerce Store,Neither,Computer / Telecom / Electronics,5 to 10,Marketing - Strategy,To attend sessions,NA,NA,Not Applicable - I am not an Affiliate/Publisher
5109,https://affiliatesummit.app.swapcard.com/event...,Attendee TypeAffiliate/Publisher,Affiliate/Publisher,NA,NA,501 to 1000,NA,To network,https://www.linkedin.com/in/amyzuspan,NA,Coupons
5110,https://affiliatesummit.app.swapcard.com/event...,Attendee TypeAffiliate/Publisher,Affiliate/Publisher,NA,NA,1 (it's just me),NA,To network,NA,NA,Media Buyer
